In [0]:
dbutils.fs.ls('/databricks-datasets')

In [0]:
%python
# List files in iot directory (JSON)
display(dbutils.fs.ls("dbfs:/databricks-datasets/iot/"))

# List files in wine-quality directory (CSV)
display(dbutils.fs.ls("dbfs:/databricks-datasets/wine-quality/"))

In [0]:
%sql

SELECT * FROM json.`dbfs:/databricks-datasets/iot/iot_devices.json`
LIMIT 10;

-- Use multilines = true if the JSON is multi-line.

In [0]:
%python

IOT_FILE = "dbfs:/databricks-datasets/iot/iot_devices.json"

IOT_FILE_PATH = "dbfs:/databricks-datasets/iot/"

dbutils.widgets.text("iot_file_path", IOT_FILE_PATH, "IoT File Path")

In [0]:
%sql

-- Create a Delta table in the default catalog
-- Unity Catalog Restrictions: Unity Catalog manages table metadata centrally and requires storage to be associated with governed external locations (e.g., cloud storage like Azure Data Lake Storage, AWS S3, or Google Cloud Storage) or managed locations. The dbfs:/ scheme is not supported for table storage in Unity Catalog because it’s tied to the Databricks File System (DBFS), which is not fully governed.
-- If you don’t specify a LOCATION for a Delta table in Unity Catalog, the data is stored in a managed location controlled by Unity Catalog

CREATE TABLE IF NOT EXISTS default.iot_data
USING DELTA
-- LOCATION 'dbfs:/user/iot_data_delta'
AS
SELECT * FROM json.`${iot_file_path}/iot_devices.json`
LIMIT 10;

In [0]:
%sql

CREATE OR REPLACE TEMP VIEW iot_data_update AS 
SELECT * FROM json.`:iot_file_path/iot_devices.json`
LIMIT 100;

-- -- Merge
-- MERGE INTO default.iot_data AS target
-- USING iot_data_update AS source
-- ON target.device_id = source.device_id
-- WHEN MATCHED AND target.device_name IS NOT NULL THEN
--   UPDATE SET
--     target.device_name = source.device_name,
--     target.humidity = source.humidity
-- WHEN NOT MATCHED THEN INSERT *
-- ;

-- Merge
MERGE INTO default.iot_data AS target
USING iot_data_update AS source
ON target.device_id = source.device_id
WHEN MATCHED AND target.device_name IS NOT NULL THEN
  UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
;


  -- INSERT (
  --   device_id,
  --   timestamp,
  --   value
  -- )
  -- VALUES (
  --   source.device_id,
  --   source.timestamp,
  --   source.value
  -- );